In [1]:
%load_ext dotenv
%dotenv

In [2]:
from google.cloud import bigquery

In [4]:
sql = """
select 
b.title_type as show_type,
b.primary_title as show_title,
b.start_year as show_start_year,
b.end_year as show_end_year,
b.genre1 as show_genre_top,
e.parent_title_id as show_id,
e.title_id as episode_id,
e.season_number as episode_season_number,
et.primary_title as episode_title,
et.start_year as episode_year,
et.runtime_minutes as episode_runtime_minutes,
e.episode_number as episode_number, r.average_rating as episode_rating,
r.num_votes as episode_rating_votes,
row_number()over(partition by e.parent_title_id order by e.season_number, e.episode_number) as show_episode_number
from `dw-realdata.imdb.title_episode` as e
join `dw-realdata.imdb.title_ratings` as r on r.title_id = e.title_id
join `dw-realdata.imdb.title_basics` as b on b.title_id = e.parent_title_id
join `dw-realdata.imdb.title_basics` as et on et.title_id = e.title_id
"""

In [5]:
client = bigquery.Client()
df = client.query(sql).to_dataframe()
df.to_csv("local-files/imdb_show_episode_ratings.csv", index=False)